Make simple snn to classify MNIST

In [1]:
import os
import time
import argparse
import sys
import datetime
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
from torch.cuda import amp
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from spikingjelly.activation_based import lava_exchange


from spikingjelly.activation_based import neuron, layer, functional, monitor
from spikingjelly.activation_based import surrogate, encoding

device = None
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)

cuda


Define Model

In [2]:
class SNN(nn.Module):
    def __init__(self, tau):
        super().__init__()
        self.layer1 = nn.Sequential(
            layer.Linear(28*28, 10, bias=False),
            neuron.LIFNode(tau=tau, surrogate_function=surrogate.ATan())
        )
    def forward(self, x: torch.Tensor):
        # print(x.shape)
        return self.layer1(x)

Create a model with a default tau value

In [3]:
tau = 2.0 #membrane time constant. 
#^ A default value in some of the spikingjelly repo files.
model = SNN(tau=tau).to(device=device)

Download the MNIST dataset and make train and test datasets

In [4]:
data_dir='./data'
train_dataset = torchvision.datasets.MNIST(
    root=data_dir,
    train=True,
    transform=torchvision.transforms.ToTensor(),
    download=True
)
test_dataset = torchvision.datasets.MNIST(
    root=data_dir,
    train=False,
    transform=torchvision.transforms.ToTensor(),
    download=True
)

In [5]:
batch_size=32
num_workers=10 #todo: see if this can be increased with more cores
train_data_loader = data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=num_workers,
    pin_memory=True
)
test_data_loader = data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers,
    pin_memory=True
)

More settings

In [6]:
EPOCHS=50
AMP=True #automatic mixed precision training
lr= 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
out_dir = "./outputs/SLM"
encoder = encoding.PoissonEncoder()
timesteps = 25

Make settings for AMP and outputs

In [7]:
scaler = None
if AMP:
    scaler = amp.GradScaler()
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    print(f'Mkdir {out_dir}.')

writer = SummaryWriter(out_dir, purge_step=0)
with open(os.path.join(out_dir, 'args.txt'), 'w', encoding='utf-8') as args_txt:
    args_txt.write('\n')
    args_txt.write(' '.join(sys.argv))

Start training

In [8]:
functional.reset_net(model)
for epoch in range(EPOCHS):
    start_time = time.time()
    model.train()
    train_loss = 0
    train_acc = 0
    train_samples = 0
    for x, label in train_data_loader:
        x = x.to(device)
        x = torch.reshape(x, (batch_size, -1))
        label = label.to(device)
        label_onehot = F.one_hot(label, 10).float()

        if scaler is None:
            out_fr = 0.
            for t in range(timesteps):
                encoded_img = encoder(x)
                out_fr += model(encoded_img)
            out_fr = out_fr / timesteps
            loss = F.mse_loss(out_fr, label_onehot)
            loss.backward()
            optimizer.step()
        else:
            with amp.autocast():
                out_fr = 0.
                for t in range(timesteps):
                    encoded_img = encoder(x)
                    out_fr += model(encoded_img)
                out_fr = out_fr / timesteps
                loss = F.mse_loss(out_fr, label_onehot)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        
        train_samples += label.numel()
        train_loss += loss.item() * label.numel()
        train_acc += (out_fr.argmax(1) == label).float().sum().item()

        functional.reset_net(model) #need to reset the snn before reuse
    print('epoch: ' + str(epoch) + '; loss' + str(train_loss))

Save the SNN Model so that you do not have to retrain

In [ ]:
input('are you sure you want to save the moel?')
file_dir = './Models/'
if not os.path.exists(file_dir):
    os.makedirs(file_dir)
    print(f'Mkdir {file_dir}.')
full_path = file_dir + '/SingleLayer_SNN.pt'
torch.save(model.state_dict(), f=full_path)

Load Model

In [ ]:
file_dir = './Models/'
full_path = file_dir + '/SingleLayer_SNN.pt'
checkpoint = torch.load(f=full_path)
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [ ]:
#test the accuracy
model.eval()
test_loss = 0
test_acc = 0
test_samples = 0
with torch.no_grad():
    for x, label in test_data_loader:
        x = x.to(device)
        # print(x.shape)
        x = torch.reshape(x, (x.shape[0], -1))
        label = label.to(device)
        label_onehot = F.one_hot(label, 10).float()
        out_fr = 0.
        for t in range(timesteps):
            encoded_img = encoder(x)
            out_fr += model(encoded_img)
        out_fr = out_fr / timesteps
        loss = F.mse_loss(out_fr, label_onehot)

        test_samples += label.numel()
        test_loss += loss.item() * label.numel()
        test_acc += (out_fr.argmax(1) == label).float().sum().item()
        functional.reset_net(model)
test_time = time.time()
# test_speed = test_samples / (test_time - train_time)
test_loss /= test_samples
test_acc /= test_samples
# writer.add_scalar('test_loss', test_loss, epoch)
# writer.add_scalar('test_acc', test_acc, epoch)
print(test_acc)

0.9122


Get Spike Counts

In [ ]:
spike_monitor = monitor.OutputMonitor(model, neuron.LIFNode)

In [ ]:
spike_monitor = monitor.OutputMonitor(model, neuron.LIFNode)
#test the accuracy
start_time = time.time()
model.eval()
test_loss = 0
test_acc = 0
test_samples = 0
z = 0
with torch.no_grad():
    for x, label in test_data_loader:
        x = x.to(device)
        # print(x.shape)
        x = torch.reshape(x, (x.shape[0], -1))
        label = label.to(device)
        label_onehot = F.one_hot(label, 10).float()
        out_fr = 0.
        for t in range(timesteps):
            encoded_img = encoder(x)
            out_fr += model(encoded_img)
        out_fr = out_fr / timesteps
        loss = F.mse_loss(out_fr, label_onehot)

        test_samples += label.numel()
        test_loss += loss.item() * label.numel()
        test_acc += (out_fr.argmax(1) == label).float().sum().item()
        functional.reset_net(model)
        # if z > 1:
        #     break
        # z += 1
test_time = time.time()
# test_speed = test_samples / (test_time - train_time)
test_loss /= test_samples
test_acc /= test_samples
# writer.add_scalar('test_loss', test_loss, epoch)
# writer.add_scalar('test_acc', test_acc, epoch)
print(test_acc)
print(test_samples)
print(model)
total_spikes = 0
for tens in spike_monitor.records:
    tensnp = tens.cpu().numpy()
    total_spikes += np.sum(tensnp)
print('total spikes: ' + str(total_spikes))
print(f'spike_seq_monitor.records=\n{len(spike_monitor.records)}')
print('total time steps: ' + str(timesteps*test_samples))

0.9124
10000
SNN(
  (layer1): Sequential(
    (0): Linear(in_features=784, out_features=10, bias=False)
    (1): LIFNode(
      v_threshold=1.0, v_reset=0.0, detach_reset=False, step_mode=s, backend=torch, tau=2.0
      (surrogate_function): ATan(alpha=2.0, spiking=True)
    )
  )
)
total spikes: 68111.0
spike_seq_monitor.records=
2191
total time steps: 70000


Calculate Power

In [ ]:
neurons = 10
Pi = 0.25
Pb = 4/1000
Pn = 0.0234 #watts (J per second)
Ps = 11.3 * (10**-9)

power = Pi + Pb + (neurons * Pn) + (total_spikes * Ps)
print('total power (watts): ' + str(power))

total power (watts): 0.48876965429999997


In [ ]:
print(spike_monitor.records[0].shape)
print(len(spike_monitor.records))
print(f'spike_seq_monitor.monitored_layers={spike_monitor.monitored_layers}')
for p in model.parameters():
    print(p.shape)
    print(len(p))

torch.Size([32, 10])
2191
spike_seq_monitor.monitored_layers=['layer1.1']
torch.Size([10, 784])
10


Visualize the output

In [ ]:
batch_size=1
num_workers=10 #todo: see if this can be increased with more cores
train_data_loader = data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=num_workers,
    pin_memory=True
)
test_data_loader = data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers,
    pin_memory=True
)

In [ ]:
spike_monitor = monitor.OutputMonitor(model, neuron.LIFNode)
#test the accuracy
start_time = time.time()
model.eval()
test_loss = 0
test_acc = 0
test_samples = 0
out_list = []
in_list = []
v_list = []
image_data = 0
with torch.no_grad():
    for x, label in test_data_loader:
        image_data = x.numpy()
        x = x.to(device)
        x = torch.reshape(x, (x.shape[0], -1))
        label = label.to(device)
        label_onehot = F.one_hot(label, 10).float()
        out_fr = 0.
        for t in range(timesteps):
            encoded_img = encoder(x)
            in_list.append(encoded_img.cpu().squeeze(0).numpy())
            out = model(encoded_img)
            out_list.append(out.cpu().squeeze(0).numpy())
            out_fr += out
            v_list.append((model.layer1[1].v).cpu().squeeze(0).numpy())
        out_fr = out_fr / timesteps
        print(out_fr)
        functional.reset_net(model)
        break


print(in_list[0].shape)
print(v_list[0].shape)
print(out_list[0].shape)

in_np = np.stack(in_list, axis=0) #axis 0 should be the timestep
v_np = np.stack(v_list, axis=0) #axis 0 should be the timestep
out_np = np.stack(out_list, axis=0) #axis 0 should be the timestep
print(in_np.shape)
print(v_np.shape)
print(out_np.shape)
print(np.squeeze(image_data).shape)
np.savetxt(fname="./Output_Image_Data/input_spikes.txt", X=in_np)
np.savetxt(fname="./Output_Image_Data/voltages.txt", X=v_np)
np.savetxt(fname="./Output_Image_Data/output_spikes.txt", X=out_np)
np.savetxt(fname="./Output_Image_Data/img_data.txt", X=np.squeeze(image_data))
# f_in = open("./Output_Image_Data/input_spikes.txt", 'x')
# f_v = open("./Output_Image_Data/voltages.txt", 'x')
# f_out = open("./Output_Image_Data/output_spikes.txt", 'x')
# f_im = open("./Output_Image_Data/img_data.txt", 'x')


tensor([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]], device='cuda:0')
(784,)
(10,)
(10,)
(7, 784)
(7, 10)
(7, 10)
(28, 28)
